# Capítulo 01: Introdução ao DuckDB

DuckDB é um banco de dados SQL analítico otimizado para análise OLAP, funcionando in-process sem necessidade de servidor externo. Ideal para ciência de dados, pipelines ETL e análise de grandes volumes de dados.

## 📦 Instalação e Configuração Inicial

Instalação das dependências necessárias para trabalhar com DuckDB e extensão Delta.

In [ ]:
# Instalação de dependências
%pip install duckdb deltalake pandas pyarrow -q

## 🔧 Setup DuckDB com Funções Utilitárias

Criação de funções helper para instalação segura de extensões.

In [ ]:
import duckdb
import importlib.util

def has_module(name):
    """Verifica se módulo Python está instalado"""
    return importlib.util.find_spec(name) is not None

def safe_install_ext(con, ext_name):
    """Instala e carrega extensão DuckDB de forma segura"""
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✓ Extensão '{ext_name}' carregada com sucesso")
        return True
    except Exception as e:
        print(f"⚠ Falha ao carregar '{ext_name}': {e}")
        return False

# Criar conexão em memória
con = duckdb.connect(database=':memory:')
print(f"DuckDB versão: {duckdb.__version__}")

## 📊 Exemplo 1: Análise de CSV

Leitura e análise de arquivo CSV com agregações básicas.

In [ ]:
# Analisar arquivo CSV com agregações
result = con.execute("""
    SELECT
        region,
        AVG(sales) as avg_sales,
        COUNT(*) as total_orders
    FROM read_csv('sales_data.csv')
    GROUP BY region
    ORDER BY avg_sales DESC
""").fetchall()

print("Vendas por Região:")
for row in result:
    print(f"  {row[0]}: média ${row[1]:.2f}, {row[2]} pedidos")

## 📤 Exemplo 2: Transformação e Exportação

Filtrar dados JSON e exportar para formato Parquet.

In [ ]:
# Transformar JSON para Parquet com filtro
con.execute("""
    COPY (
        SELECT * 
        FROM read_json('input.json')
        WHERE date >= '2024-01-01'
    ) TO 'output.parquet' (FORMAT PARQUET)
""")

print("✓ Dados transformados e exportados para output.parquet")

## 🔄 Exemplo 3: Trabalhando com Delta Lake

Análise avançada de tabelas Delta com agregações e exportação.

In [ ]:
# Análise complexa em tabela Delta
result = con.execute("""
    -- 1. Criar view sobre dados Delta
    CREATE OR REPLACE VIEW sales AS
    SELECT * FROM delta_scan('./delta_tables/sales');

    -- 2. Análise agregada por mês e categoria
    SELECT
        DATE_TRUNC('month', order_date) as month,
        product_category,
        SUM(revenue) as total_revenue,
        AVG(revenue) as avg_revenue,
        COUNT(DISTINCT customer_id) as unique_customers
    FROM sales
    WHERE order_date >= '2024-01-01'
    GROUP BY 1, 2
    HAVING SUM(revenue) > 10000
    ORDER BY 1 DESC, 3 DESC
""").fetchall()

print("📊 Vendas por Mês e Categoria:")
for row in result:
    print(f"  {row[0]} - {row[1]}: ${row[2]:,.2f} ({row[4]} clientes)")

In [ ]:
# Exportar resultados para formato Parquet com compressão
con.execute("""
    COPY (
        SELECT * FROM sales 
        WHERE region = 'North America'
    ) TO 'na_sales.parquet' (FORMAT PARQUET, COMPRESSION ZSTD)
""")

print("✓ Dados exportados com compressão ZSTD")

# Capitulo 01 Introducao Ao Duckdb

Notebook gerado automaticamente a partir do código fonte python.
